In [1]:
from pathlib import Path
from urllib.request import urlretrieve
from zipfile import ZipFile
import glob
import pandas as pd

### Data Acquisition

In [2]:
#Identify pattern of the url associated with each month's data file and code accordingly.

url_front = "https://s3.amazonaws.com/tripdata/JC-"
url_end = "-citibike-tripdata.csv.zip"

filename_base = Path('/Users/xuan/Documents/Python/portfolio_projects/ds_citibike')

In [3]:
def get_year_month_files(list_of_year_month):
    '''
    Main function to retrieve files and store them locally.
    '''
    for year_month in list_of_year_month:
        url = url_front + year_month + url_end
        filename = year_month + '.zip'
        filepath = filename_base / filename
        path, headers = urlretrieve(url, filepath)

In [4]:
#Create a function that helps with generating the year-month list needed in the main function.

def combine_year_month(year, list_of_months):
    '''
    Generates combined strings of year and month.
    Parameters are a string value of 4-digit year and a list with 2-digit, string month numbers.
    '''
    list_str_year_month = []
    for i in list_of_months:
        str_year_month = year + i
        list_str_year_month.append(str_year_month)
    return list_str_year_month        

In [5]:
#Include the relevant months.

list_of_2022_months = ['08', '09', '10', '11', '12']
list_of_2023_months = ['01', '02', '03', '04', '05', '06', '07']

In [6]:
#Call the function to get all year-month in one place.

all_year_month = combine_year_month('2022', list_of_2022_months) + combine_year_month('2023', list_of_2023_months)
print(all_year_month)

['202208', '202209', '202210', '202211', '202212', '202301', '202302', '202303', '202304', '202305', '202306', '202307']


In [7]:
#Call the main function to get all files.

get_year_month_files(all_year_month)

In [8]:
#Unzip the files

def unzip_files(list_of_year_month):
    '''
    Extract corresponding csv files for each year and month.
    '''
    for year_month in list_of_year_month:
        filename = year_month + '.zip'
        filepath = filename_base / filename
        with ZipFile(filepath, 'r') as f:
            list_of_names = f.namelist()
            for name in list_of_names:
                if name.endswith('.csv'):
                    f.extract(name)

In [9]:
unzip_files(all_year_month)

In [44]:
#Aggregate data files

trip_files = glob.glob('JC-20*.csv')

df_list = []

for file in trip_files:
    data = pd.read_csv(file)
    df_list.append(data)

trips = pd.concat(df_list)

### Data Cleaning

In [51]:
#Preview data. Data is tidy.

trips.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,557150C42CBD584B,classic_bike,2023-02-16 18:05:38,2023-02-16 18:13:47,Mama Johnson Field - 4 St & Jackson St,HB404,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,40.743190,-74.040126,40.736982,-74.027781,member
1,9F1B2C0BCD476C66,classic_bike,2023-02-15 19:20:33,2023-02-15 19:27:06,Mama Johnson Field - 4 St & Jackson St,HB404,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,40.743222,-74.040081,40.736982,-74.027781,member
2,4A4A29A7C1B54278,classic_bike,2023-02-18 14:58:16,2023-02-18 15:06:15,Mama Johnson Field - 4 St & Jackson St,HB404,12 St & Sinatra Dr N,HB201,40.743140,-74.040041,40.750604,-74.024020,casual
3,BBF4E1D39D41D78C,classic_bike,2023-02-24 15:16:12,2023-02-24 15:24:17,Mama Johnson Field - 4 St & Jackson St,HB404,Mama Johnson Field - 4 St & Jackson St,HB404,40.743187,-74.040179,40.743140,-74.040041,member
4,3CECAA17C4ABBF30,classic_bike,2023-02-16 18:05:02,2023-02-16 18:05:04,Mama Johnson Field - 4 St & Jackson St,HB404,Mama Johnson Field - 4 St & Jackson St,HB404,40.743172,-74.040054,40.743140,-74.040041,member


In [52]:
print(len(trips))

981011


In [53]:
trips = trips.drop_duplicates()

In [54]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981011 entries, 0 to 48584
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             981011 non-null  object 
 1   rideable_type       981011 non-null  object 
 2   started_at          981011 non-null  object 
 3   ended_at            981011 non-null  object 
 4   start_station_name  980936 non-null  object 
 5   start_station_id    980936 non-null  object 
 6   end_station_name    977641 non-null  object 
 7   end_station_id      977641 non-null  object 
 8   start_lat           981011 non-null  float64
 9   start_lng           981011 non-null  float64
 10  end_lat             979712 non-null  float64
 11  end_lng             979712 non-null  float64
 12  member_casual       981011 non-null  object 
dtypes: float64(4), object(9)
memory usage: 104.8+ MB
